In [3]:
import pandas as pd
import sys  
sys.path.insert(1, '/home/tb24/projects/llm-data-aug')

# Path
import os

# Get the project root directory (one level up from the notebook directory)
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Define the data path
data_path = os.path.join(project_root, "data")

df = pd.read_csv("/home/tb24/projects/llm-data-aug/data/llm_generated/negtative_user_reviews.csv")
df


,0,1,2
0,"('reviews', ['Hamburger có vị tệ.', 'Đồ ăn nha...","('sentiments', [0, 0, 0, 1, 1])","('n', 5)"
1,"('reviews', [""Tôi vừa có một trải nghiệm kinh ...","('sentiments', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0])","('n', 10)"
2,"('reviews', [""Tôi vừa có một trải nghiệm kinh ...","('sentiments', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0])","('n', 10)"
3,"('reviews', [""Tôi đã có một trải nghiệm kinh k...","('sentiments', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0])","('n', 10)"
4,"('reviews', [""Tôi vừa trải qua một trải nghiệm...","('sentiments', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0])","('n', 10)"
...,...,...,...
1375,"('reviews', ['Hamburger có vị ㅠㅠ.', 'Đồ ăn nha...","('sentiments', [2, 2, 2, 2, 2])","('n', 5)"
1376,"('reviews', [""Tôi vừa có một trải nghiệm kinh ...","('sentiments', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0])","('n', 10)"
1377,"('reviews', ['Hamburger có vị ㅠㅠ.', 'Đồ ăn nha...","('sentiments', [2, 2, 2, 2, 2, 0, 0, 0, 0, 0])","('n', 10)"
1378,"('reviews', ['Hamburger có vị chán quá ㅠㅠ.', '...","('sentiments', [0, 0, 0, 1, 1, 0, 0, 0, 0, 0])","('n', 10)"


In [2]:
# Transform the data to the format of UserReview
from synthesizer.models import UserReview

user_reviews = []
reviews = []
sentiments = []
for row in df.to_dict(orient="records"):
    _reviews = row.get("0")
    _reviews = _reviews[1:-1].replace("'", "").replace("reviews, ", "")[1:-1]
    # To a list
    _reviews = _reviews.split(", ")
    reviews.extend(_reviews)

    _sentiments = row.get("1")
    _sentiments = _sentiments[1:-1].replace("'", "").replace("sentiments, ", "")[1:-1]
    # To a list
    _sentiments = _sentiments.split(", ")
    sentiments.extend(_sentiments)
    n = row.get("2")
    n = n[1:-1].replace("'", "").replace("n, ", "")
    # n = n.split(", ")

for review, sentiment in zip(reviews, sentiments):
    tmp = UserReview(review=review, sentiment=sentiment)
    user_reviews.append(tmp)

print(len(user_reviews))


11713


In [ ]:
from synthesizer.models import UserReviews

user_reviews = UserReviews(reviews=user_reviews, batch_size=10)
user_reviews

In [ ]:
# TO a dataframe
user_reviews_df = pd.DataFrame.from_dict(user_reviews.model_dump(), orient="columns")
user_reviews_df

# Save the dataframe to a csv file
# user_reviews_df.to_csv("/home/tb24/projects/llm-data-aug/data/llm_generated/negtative_user_reviews.csv", index=False)


In [ ]:
# Unpack the data:
# - reviews -> review_1, review_2, ...
# - sentiments -> sentiment_1, sentiment_2, ...
# - n -> n_1, n_2, ...

# Create a new dataframe with the unpacked data


user_reviews_df = 